In [136]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import csv
import os

# training
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [137]:

DATA_PATH = os.path.join('MP_Data')  # Path for exported data, numpy arrays


#getting dataset values from csv
with open('newlabelstrain.csv', newline='') as f:
    reader = csv.reader(f)
    data = [row[0] for row in reader]


actions = np.array(data)
no_sequences = 30  # thirty videos worth of data
sequence_length = 30  # each video is 30 frame in length

label_map = {label:num for num, label in enumerate(actions)}
print(label_map)


{'hello': 0, 'goodMorning': 1, 'sir': 2}


In [138]:

sequences, labels = [], []  # x data, y data
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

x = np.array(sequences)
print(x.shape)
y = to_categorical(labels).astype(int)
print(y)


(90, 30, 1662)
[[1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [139]:

# creating training partitions
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(85, 30, 1662) (5, 30, 1662) (85, 3) (5, 3)


In [140]:

# training

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential() # instantiating the model
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [141]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])
print(model.summary())

Epoch 1/2000
3/3 [==============================] - 3s 215ms/step - loss: 12.0825 - categorical_accuracy: 0.3294
Epoch 2/2000
3/3 [==============================] - 0s 58ms/step - loss: 9.0015 - categorical_accuracy: 0.3294
Epoch 3/2000
3/3 [==============================] - 0s 58ms/step - loss: 14.5960 - categorical_accuracy: 0.3176
Epoch 4/2000
3/3 [==============================] - 0s 58ms/step - loss: 3.0249 - categorical_accuracy: 0.4471
Epoch 5/2000
3/3 [==============================] - 0s 58ms/step - loss: 12.6190 - categorical_accuracy: 0.2706
Epoch 6/2000
3/3 [==============================] - 0s 57ms/step - loss: 14.6074 - categorical_accuracy: 0.3529
Epoch 7/2000
3/3 [==============================] - 0s 60ms/step - loss: 11.7714 - categorical_accuracy: 0.4118
Epoch 8/2000
3/3 [==============================] - 0s 58ms/step - loss: 44.9449 - categorical_accuracy: 0.4706
Epoch 9/2000
3/3 [==============================] - 0s 59ms/step - loss: 5.8215 - categorical_accuracy: 0

3/3 [==============================] - 0s 66ms/step - loss: 0.9576 - categorical_accuracy: 0.5412
Epoch 74/2000
3/3 [==============================] - 0s 73ms/step - loss: 0.9196 - categorical_accuracy: 0.4824
Epoch 75/2000
3/3 [==============================] - 0s 57ms/step - loss: 0.9097 - categorical_accuracy: 0.5647
Epoch 76/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.9113 - categorical_accuracy: 0.6706
Epoch 77/2000
3/3 [==============================] - 0s 61ms/step - loss: 0.8869 - categorical_accuracy: 0.6235
Epoch 78/2000
3/3 [==============================] - 0s 59ms/step - loss: 0.8659 - categorical_accuracy: 0.6588
Epoch 79/2000
3/3 [==============================] - 0s 57ms/step - loss: 0.8565 - categorical_accuracy: 0.7412
Epoch 80/2000
3/3 [==============================] - 0s 65ms/step - loss: 0.8589 - categorical_accuracy: 0.6353
Epoch 81/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.8440 - categorical_accuracy: 0.6706
Epoch 

3/3 [==============================] - 0s 62ms/step - loss: 0.5355 - categorical_accuracy: 0.7529
Epoch 146/2000
3/3 [==============================] - 0s 59ms/step - loss: 0.5309 - categorical_accuracy: 0.7647
Epoch 147/2000
3/3 [==============================] - 0s 59ms/step - loss: 0.5215 - categorical_accuracy: 0.8000
Epoch 148/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.5202 - categorical_accuracy: 0.7529
Epoch 149/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.5142 - categorical_accuracy: 0.7882
Epoch 150/2000
3/3 [==============================] - 0s 58ms/step - loss: 0.5104 - categorical_accuracy: 0.7765
Epoch 151/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.5065 - categorical_accuracy: 0.8000
Epoch 152/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.5119 - categorical_accuracy: 0.7882
Epoch 153/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.5004 - categorical_accuracy: 0.788

3/3 [==============================] - 0s 63ms/step - loss: 0.5240 - categorical_accuracy: 0.6588
Epoch 218/2000
3/3 [==============================] - 0s 60ms/step - loss: 0.4937 - categorical_accuracy: 0.6941
Epoch 219/2000
3/3 [==============================] - 0s 69ms/step - loss: 0.5316 - categorical_accuracy: 0.7647
Epoch 220/2000
3/3 [==============================] - 0s 59ms/step - loss: 0.5214 - categorical_accuracy: 0.6706
Epoch 221/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.5112 - categorical_accuracy: 0.7529
Epoch 222/2000
3/3 [==============================] - 0s 59ms/step - loss: 0.4740 - categorical_accuracy: 0.7647
Epoch 223/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.4983 - categorical_accuracy: 0.6706
Epoch 224/2000
3/3 [==============================] - 0s 70ms/step - loss: 0.4800 - categorical_accuracy: 0.7647
Epoch 225/2000
3/3 [==============================] - 0s 60ms/step - loss: 0.4839 - categorical_accuracy: 0.729

KeyboardInterrupt: 

In [142]:
res = model.predict(x_test)
print(res)
print(actions[np.argmax(res[0])])
for i in range(len(res)):
    print(actions[np.argmax(res[i])])

[[0.01170408 0.08003964 0.9082563 ]
 [0.39746284 0.5917595  0.01077764]
 [0.1759892  0.7853977  0.0386132 ]
 [0.2682868  0.72802275 0.00369045]
 [0.00244975 0.03708689 0.96046335]]
sir
sir
goodMorning
goodMorning
goodMorning
sir


In [143]:
# save weights
model.save('action.h5')

In [144]:
#evaluation using confusion matrix
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

yhat = model.predict(x_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
matrix = multilabel_confusion_matrix(ytrue, yhat)
print(matrix)
accuracy_score = accuracy_score(ytrue, yhat)
print(f"{accuracy_score= }")


[[[2 0]
  [3 0]]

 [[2 3]
  [0 0]]

 [[3 0]
  [0 2]]]
accuracy_score= 0.4
